1. ISI
2. Templates overtime 
3. All templates
4. UMAP overtime
5. Features overtime (Pointplots)

In [ ]:
import matplotlib
matplotlib.use('Agg')

import glob
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd
import spikeinterface.core as sc
import spikeinterface.curation as scu
import spikeinterface.extractors as se
import spikeinterface.widgets as sw
import sys 

from tqdm.auto import tqdm

sys.path.append('src')

from utils import plot_isi_by_session

sorted_date = '20231229'
mice = sorted([mouse_path.split(os.sep)[-1] for mouse_path in glob.glob(f'data{os.sep}sorted{os.sep}{sorted_date}{os.sep}**')])
print(f'Plotting mice: {mice} sorted on {sorted_date}')

In [ ]:
for mouse in (pbar := tqdm(mice)):
    mouse_sorted_folder = f'data{os.sep}sorted{os.sep}{sorted_date}{os.sep}{mouse}'
    mouse_isi_folder = f'{mouse_sorted_folder}{os.sep}isi'
    os.makedirs(mouse_isi_folder, exist_ok=True)

    recording_processed = sc.load_extractor(f'{mouse_sorted_folder}{os.sep}processed')

    sorting = se.NpzSortingExtractor(f'{mouse_sorted_folder}{os.sep}sorting{os.sep}sorter_output{os.sep}firings.npz')
    # spikeinterface https://github.com/SpikeInterface/spikeinterface/pull/1378
    sorting = scu.remove_excess_spikes(sorting, recording_processed)

    waveform_extractor = sc.load_waveforms(
        folder=f'{mouse_sorted_folder}{os.sep}waveforms', with_recording=True, sorting=sorting
    )
    extremum_channels = sc.get_template_extremum_channel(waveform_extractor, peak_sign='neg')

    sessions = pd.read_csv(f'{mouse_sorted_folder}{os.sep}sessions.csv').sort_values(by='date')

    for unit_id in sorting.unit_ids:
        pbar.set_description(f'{mouse} -> [{unit_id} / {len(sorting.unit_ids)}]')
        unit_isi_plot_file = f'{mouse_isi_folder}{os.sep}{unit_id}.png'
        if not os.path.isfile(unit_isi_plot_file):
            plot_isi_by_session(sorting, unit_id, sessions, savepath=unit_isi_plot_file)